#How To Build a Model: Step By Step


In [2]:
#Pytorch Imports
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.datasets import IMDB
from torchtext.transforms import ToTensor
from torchtext.vocab import build_vocab_from_iterator
#Pandas
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import time




In [3]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from transformers import AutoTokenizer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

Pick Your Universal Hyperparamaters:

In [4]:
batch_size = 1000000
epochs = 3
input_dim = 512
hidden_dim = 100
output_dim = 10
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"==>> torch.cuda.is_available(): {torch.cuda.is_available()}")

==>> torch.cuda.is_available(): False


# Setup Your Model Workspace:

Your model architecture, and all of it's relevent code, will go in a folder at the location ./NNDL/(Your Model Name Here)


# Setup Your Dataset/Data Loaders Here

Pytorch usually takes a csv file. You need to write a function( For naming convention let's call it *create_torch_datasets()* ) that takes the universal dataset kyle got from parsing and seperates/loads it into data that you want your model to train on




In [5]:
#Create Custom Dataset Class:
class TextDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.root_dir = root_dir
        self.data = pd.read_csv(csv_file)
        self.mapping = {}
        self.cur_map = 1
        self.tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        text_o = os.path.join(self.root_dir,
            self.data.iloc[index, 0])
        tokens = self.tokenizer(text_o, padding='max_length',truncation=True)['input_ids']
        # print(f"==>> tokens: {tokens}")
        # token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        # print(f"==>> token_ids: {token_ids}")
        
        #input_tensor = torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)
        category_text = os.path.join(self.root_dir,
            self.data.iloc[index, 1])
        if category_text in self.mapping:
            category = self.mapping[category_text]
        else:
            self.mapping[category_text] = self.cur_map
            category = self.cur_map
            self.cur_map+=1
        # print(f"self.mapping:{self.mapping}")
        return torch.tensor(tokens,dtype=torch.float32),category

In [6]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "Hello, how are you doing?"

# Tokenize the text
tokens = tokenizer.tokenize(text)

# Convert tokens to numerical IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Print the results
print("Original Text:", text)
print("Tokens:", tokens)
print("Token IDs:", torch.tensor(token_ids))

Original Text: Hello, how are you doing?
Tokens: ['hello', ',', 'how', 'are', 'you', 'doing', '?']
Token IDs: tensor([7592, 1010, 2129, 2024, 2017, 2725, 1029])


In [7]:
#This is an example of what I'm talking about. The Fashion dataset is pretty easy bc it's function alr exists
# but you need to create your own function to make the training data and test_ data datasets
text_dataset = TextDataset(csv_file=r'data\products_noimg.csv',root_dir='/')
# for i, sample in enumerate(text_dataset):
#     print(i, len(sample[0]), sample[1])
generator1 = torch.Generator().manual_seed(42)
train,test = torch.utils.data.random_split(text_dataset,[0.8,0.2], generator=generator1)
train_dataloader =  DataLoader(train, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
test_dataloader =  DataLoader(test, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)

# train_iter= IMDB(split="train")
# test_iter = IMDB(split="test")
# tokenizer = get_tokenizer("basic_english")
# def yield_tokens(data_iter):
#     for _, text in data_iter:
#         yield tokenizer(text)
# train_tokens = []
# vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
# vocab.set_default_index(vocab["<unk>"])
# test_tokens = []
# test_tokens = build_vocab_from_iterator(yield_tokens(test_iter), specials=["<unk>"])



Put the dataset into data loader, and check the shape, make sure it's how you want it.


In [8]:
# Create data loaders.
#train_dataloader = DataLoader(text_dataset, batch_size=batch_size,shuffle=True, num_workers=0)
#print(train_dataloader[0])
# for i_batch, sample_batched in enumerate(train_dataloader):
#   print(f"==>> i_batch: {i_batch}")
#   print(f"==>> sample_batched: {len(sample_batched)}")
      #print(i_batch, sample_batched['text'][0][0].item(),
        #sample_batched['category'])

# Display image and label.
#train_features, train_labels = next(iter(train_dataloader))
# print(train_features)
# print(train_labels)
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

# Setup Model Architecture

1. Create your model architecture in your folder
2. Pick your loss function and optimizer

In [9]:
from NNDL.transformer.architecture import Transformer2
from NNDL.Utils.solver import train,test


model = nn.Transformer().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


# Train the model
This should work to train your model. We may have to make some edits for different optimizations, but we can figure it out.


In [10]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


KeyboardInterrupt: 